In [ ]:
from nbdev import *
%nbdev_default_export errors

Cells will be exported to pct.errors,
unless a different module is specified after an export flag: `%nbdev_export special.module`


# Errors

> Types of error response of a percectual control hierarchy.

In [ ]:
%nbdev_hide
import sys
sys.path.append("..")

In [ ]:
%nbdev_export
import numpy as np
from pct.hierarchy import PCTHierarchy
from abc import ABC, abstractmethod


In [ ]:
%nbdev_export
class BaseErrorType(ABC):
    "Base class of a type error response. This class is not used direclty by developers, but defines the interface common to all."
    def __init__(self):
        self.error_response=0
    
    @abstractmethod
    def __call__(self):
        pass
    
    def get_error_response(self):
        return self.error_response

In [ ]:
%nbdev_export
class RootSumSquaredError(BaseErrorType):
    "The square root of the sum of the square of the errors."
    def __init__(self):
        self.sum=0
        
    def __call__(self, error):
        self.sum+=error*error
        self.error_response=np.sqrt(self.sum)
        
    class Factory:
        def create(self): return RootSumSquaredError()

In [ ]:
%nbdev_export
class BaseErrorCollector(ABC):
    "Base class of an error collector. This class is not used direclty by developers, but defines the interface common to all."
    'Parameters:'
    'limit - the limit of valid error response'
    'error_response - the type of error response'
    
    def __init__(self, limit,error_response):
        self.limit=limit
        self.limit_exceeded=False
        self.error_response=error_response
        
    def set_limit(self, limit):
        self.limit=limit
        self.limit_exceeded=False

    def set_error_response(self, error_response):
        self.error_response=error_response
    
    def error(self):
        return self.error_response.get_error_response()
    
    def add_error_data(self, level, col, data=[]):
        for datum in data:
            self.error_response(datum)
                  
    def is_limit_exceeded(self):            
        return self.limit_exceeded        
    
    @classmethod
    def collector(cls, error_type, error_collector, limit):
        et = ErrorFactory.createError(error_type)   
        ec = ErrorFactory.createError(error_collector)   
        ec.set_limit(limit)
        ec.set_error_response(et)
        
        return ec

In [ ]:
%nbdev_export
class TotalError(BaseErrorCollector):
    "A class to collect all the errors of the control system run."            
    def __init__(self, limit=1000, error_response=None, **cargs):
        super().__init__(limit, error_response)

    def add_data(self, hpct=None):
        for level in range(len(hpct.hierarchy)):
             for col in range(len(hpct.hierarchy[level])):
                node  = hpct.hierarchy[level][col]
                self.add_error_data(level, col, [node.get_function("comparator").get_value()])
                if self.error_response.get_error_response() > self.limit:
                    self.limit_exceeded=True
                    return
    class Factory:
        def create(self): return TotalError()

In [ ]:
class ErrorFactory:
    factories = {}
    def addFactory(id, errorFactory):
        ErrorFactory.factories.put[id] = errorFactory
    addFactory = staticmethod(addFactory)
    # A Template Method:
    def createError(id):
        print(id)
        if not ErrorFactory.factories.__contains__(id):
            ErrorFactory.factories[id] = \
              eval(id + '.Factory()')
        return ErrorFactory.factories[id].create()
    createError = staticmethod(createError)

In [ ]:
er = RootSumSquaredError()
te = TotalError(error_response=er)   

In [ ]:
hpct = PCTHierarchy(1,1,error_collector=te)

hpct.run(steps=5, verbose=True)

level0col0 1.000 0.000 1.000 10.000 

level0col0 1.000 0.000 1.000 10.000 

level0col0 1.000 0.000 1.000 10.000 

level0col0 1.000 0.000 1.000 10.000 

level0col0 1.000 0.000 1.000 10.000 



10

In [ ]:
err=te.error()
print(err)

2.23606797749979


In [ ]:
et = ErrorFactory.createError('RootSumSquaredError')   
et(102)
print(et.get_error_response())

ec = ErrorFactory.createError('TotalError')   
ec.set_limit(100)
ec.set_error_response(et)
print(ec.error())

RootSumSquaredError
102.0
TotalError
102.0


In [ ]:
notebook2script()

Converted 00_examples.ipynb.
Converted 01_putils.ipynb.
Converted 02_functions.ipynb.
Converted 03_nodes.ipynb.
Converted 04_hierarchy.ipynb.
Converted 05_environments.ipynb.
Converted 06_architectures.ipynb.
Converted 07_errors.ipynb.
Converted index.ipynb.
